## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,48.00,84,0,14.65,clear sky
1,1,Nara,JP,34.6851,135.8049,48.00,81,20,8.05,few clouds
2,2,Hermanus,ZA,-34.4187,19.2345,57.00,92,6,3.11,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,44.60,70,40,25.32,scattered clouds
4,4,Calvinia,ZA,-31.4707,19.7760,70.63,24,23,2.80,few clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Calvinia,ZA,-31.4707,19.7760,70.63,24,23,2.80,few clouds
5,5,Atuona,PF,-9.8000,-139.0333,80.96,73,14,16.31,few clouds
6,6,Rumphi,MW,-11.0186,33.8575,60.44,96,53,2.80,broken clouds
7,7,Broome,US,42.2506,-75.8330,70.00,35,1,8.05,clear sky
8,8,Bud,US,39.4470,-86.1758,64.40,82,40,13.80,scattered clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                394
City                   394
Country                391
Lat                    394
Lng                    394
Max Temp               394
Humidity               394
Cloudiness             394
Wind Speed             394
Current Description    394
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                391
City                   391
Country                391
Lat                    391
Lng                    391
Max Temp               391
Humidity               391
Cloudiness             391
Wind Speed             391
Current Description    391
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Calvinia,ZA,70.63,few clouds,-31.4707,19.7760,
5,Atuona,PF,80.96,few clouds,-9.8000,-139.0333,
6,Rumphi,MW,60.44,broken clouds,-11.0186,33.8575,
7,Broome,US,70.00,clear sky,42.2506,-75.8330,
8,Bud,US,64.40,scattered clouds,39.4470,-86.1758,
9,Viedma,AR,71.60,clear sky,-40.8135,-62.9967,
13,Butaritari,KI,82.15,scattered clouds,3.0707,172.7902,
14,Saint George,US,82.00,clear sky,37.1041,-113.5841,
15,Napier,NZ,64.00,overcast clouds,-39.4833,176.9167,
16,Albany,US,72.00,scattered clouds,42.6001,-73.9662,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    #get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping")
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [43]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")

hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df.count()

City                   442
Country                442
Max Temp               442
Current Description    442
Lat                    442
Lng                    442
Hotel Name             442
dtype: int64

In [51]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hermanus,ZA,57.00,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
4,Calvinia,ZA,70.63,few clouds,-31.4707,19.7760,Die Lemoenhuis
5,Atuona,PF,80.96,few clouds,-9.8000,-139.0333,Villa Enata
6,Rumphi,MW,60.44,broken clouds,-11.0186,33.8575,Finst Rumphi Earthbag House Lodge
7,Broome,US,70.00,clear sky,42.2506,-75.8330,Chenango Valley State Park


In [48]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [81]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [1]:
# 11a. Add a marker layer for each city to the map. 

max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

NameError: name 'hotel_df' is not defined